In [35]:
import pandas as pd

from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch

import PIL
import numpy as np

In [ ]:
IMAGE_SIZE = 32
CHANNELS = 3
Z_DIM = 200
TRAIN_SPLIT = 0.8
BATCH_SIZE = 128

In [48]:
working_dir = "/home/mary/work/repos/generative_deep_Learning_2nd_edition_pytorch"
exp_dir = working_dir + "/notebooks/03_vae/03_vae_faces/"
data_dir = working_dir + "/data"
dataset_dir = data_dir + "/celeba-dataset"
image_dir = dataset_dir + "/img_align_celeba/img_align_celeba/"
labels_file = dataset_dir + "/list_attr_celeba.csv"

In [21]:
attributes = pd.read_csv(labels_file)
print(attributes.columns)
attributes.head()

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')


,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [39]:
LABEL = "Blond_Hair"
IMAGE_HEAD = "image_id"
attributes[LABEL].head()

image_list = attributes[IMAGE_HEAD].to_list()
labels_list = attributes[LABEL].to_list()

print(len(image_list), len(labels_list))
print(image_list[0], labels_list[0])

202599 202599
000001.jpg -1


In [40]:
class ImageWLabelsDataset(torch.utils.data.Dataset):
    
    def __init__(self, images_dir, images_list, labels_list, transforms):
        self.images_dir = images_dir
        self.labels = labels_list
        self.transforms = transforms
        self.img_files = images_list
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image = PIL.Image.open(self.images_dir + self.img_files[idx]).convert("RGB")
        label = torch.tensor(self.labels[idx])

        if self.transforms:
            image = self.transforms(image)
        
        return(image, label)


In [49]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), # Padding (left, top, right, bottom)
    transforms.ToTensor()
    
])

dataset = ImageWLabelsDataset(images_dir=image_dir, images_list=image_list, 
                              labels_list=labels_list, transforms=transform)



In [50]:
train_size = int(TRAIN_SPLIT * len(dataset))
test_size = int(len(dataset) - train_size)

train_data, test_data =  torch.utils.data.random_split(dataset, [train_size, test_size])

train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_data_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [ ]:
# get vector from labels
def get_vector_from_label(data_loader, vae, embedding_dim):
    curr_sum_pos = np.zeros(shape=embedding_dim, dtype=np.float32)
    curr_sum_neg = np.zeros(shape=embedding_dim, dtype=np.float32)

    curr_mean_pos = np.zeros(shape=embedding_dim, dtype=np.float32)
    curr_mean_neg = np.zeros(shape=embedding_dim, dtype=np.float32)

    curr_num_pos = 0
    curr_num_neg = 0

    curr_vector = np.zeros(shape=embedding_dim, dtype=np.float32)
    curr_dist = 0


    for batch in data_loader:
        # batch = next(data_iter)
        images = batch[0]
        attr = batch[1]

        # predict
        b = images.shape[0]
        _, _, z = vae.forward(images)
        z = torch.rand((b, embedding_dim))

        z_pos = z[attr == 1]
        z_neg = z[attr == -1]

        if len(z_pos) > 0 :
            curr_sum_pos += torch.sum(z_pos, dim=0).numpy()
            curr_num_pos += len(z_pos)
            new_mean_pos = curr_sum_pos/curr_num_pos
            movement_pos = np.linalg.norm(new_mean_pos - curr_mean_pos)

        if len(z_neg) > 0 :
            curr_sum_neg += torch.sum(z_neg, dim=0).numpy()
            curr_num_neg += len(z_neg)
            new_mean_neg = curr_sum_neg/curr_num_neg
            movement_neg = np.linalg.norm(new_mean_neg - curr_mean_neg)

        curr_vector = new_mean_pos - new_mean_neg
        new_distance = np.linalg.norm(curr_vector)
        dist_change = new_distance - curr_dist
        print("distance change = ", dist_change)

        curr_mean_pos = np.copy(new_mean_pos)
        curr_mean_neg = np.copy(new_mean_neg)
        curr_dist = new_distance

        if np.sum([movement_neg, movement_pos]) < 0.08:
            curr_vector = curr_vector / curr_dist
            print("vector found")
            break
        
    return curr_vector





    

0.9435069561004639
-0.27078754
-0.15019
-0.046568543
-0.0516842
-0.005610317
-0.04450336
-0.0155541
-0.017111927
-0.028202027
-0.018707156
-0.0074230134
-0.004292041
-0.013017058
-0.008722365
vector found


In [52]:
batch = next(data_iter)

In [55]:
batch[0].shape

torch.Size([128, 3, 32, 32])